In [1]:
%load_ext watermark
%watermark

Last updated: 2024-05-26T22:24:28.505271-05:00

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.12.2

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 165 Stepping 2, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing, feature_extraction


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing, feature_extraction
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
#import category_encoders
from sklearn.model_selection import cross_validate
from category_encoders import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.compose import ColumnTransformer

In [4]:
consumidores = pd.read_csv("../data/df_consumo_varObjetivo.csv")

In [5]:
consumidores.head()

,departamento,residentes_hogar,d2_04_num_hijos,per_edad_tipo,estrato_tipo,situacion_tipo,frecuencia_consumo_marihuana_tipo,frecuencia_consumo_cocaina_tipo,frecuencia_consumo_basuco_tipo,per_sexo_tipo,...,g_06_a_posibilidad_conseguir_marihuana_tipo,g_06_b_posibilidad_conseguir_cocaina_tipo,g_06_c_posibilidad_conseguir_basuco_tipo,g_07_alguien_ofrecio_comprar_probar_sp_tipo,g_01_a_num_familiares_consumen_sp_imp_tipo,g_02_a_num_amigos_consumen_sp_imp_tipo,g_08_a_ofrecieron_marihuana_imp_tipo,g_08_b_ofrecieron_cocaina_imp_tipo,g_08_c_ofrecieron_basuco_imp_tipo,CatRiesgo
0,Guaviare,5,0,Joven,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
1,Cordoba,4,3,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
2,Caqueta,4,2,Adulto,Estrato bajo,Trabajando,Nunca,Nunca,Nunca,Hombre,...,g_06_a_No_conoce_facilidad_dificultad,g_06_b_No_conoce_facilidad_dificultad,g_06_c_No_conoce_facilidad_dificultad,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
3,Bolivar,6,0,Adulto,Estrato alto,Trabajando,Nunca,Nunca,Nunca,Mujer,...,g_06_a_No_podria_conseguir,g_06_b_No_podria_conseguir,g_06_c_No_podria_conseguir,g_07_No,g_01_a_No_aplica,g_02_a_No_aplica,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,0
4,Tolima,4,1,Joven,Estrato bajo,Buscando trabajo,Nunca,Nunca,Nunca,Mujer,...,g_06_a_Seria_facil,g_06_b_Seria_facil,g_06_c_Seria_facil,g_07_No,g_01_a_No_aplica,g_02_a_Dos_o_mas,g_08_a_No_aplica,g_08_b_No_aplica,g_08_c_No_aplica,1


En este modelo vamos a modificar cada variable en función de su tipo. Al conjunto de pasos que siguen los datos se le llama comúnmente **Pipelines** (literalmente, sistemas de tuberias).

**INCISO**

En necesario modificar dos transformadores de scikitlearn para que sean compatibles con pipelines. Este paso es necesario en la version actual de scikit-learn

In [6]:
variable_objetivo = ["CatRiesgo"]
variables_independientes = consumidores.drop(variable_objetivo, axis=1).columns

In [7]:
datos_numericos = consumidores[variables_independientes].select_dtypes([int, float])
col_numericas = datos_numericos.columns
col_no_numericas = consumidores[variables_independientes].select_dtypes([object]).columns

dict_var_ordinales = {
    "per_edad_tipo": ["Adolescente", "Joven", "Adulto", "Persona mayor"],
    "estrato_tipo":["Estrato bajo", "Estrato medio", "Estrato alto"],
    "d_08_estado_salud_tipo":["d_08_Muy_malo","d_08_Malo","d_08_Regular","d_08_Bueno","d_08_Muy_bueno"],
    "frecuencia_consumo_marihuana_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_cocaina_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
    "frecuencia_consumo_basuco_tipo":["Nunca", "Alguna vez en su vida", "Una vez en el año", "Algunas veces durante los últimos 12 meses", "Algunas veces mensualmente", "Algunas veces semanalmente", "Diariamente"],
}

col_ordinales = list(dict_var_ordinales.keys())
datos_ordinales = consumidores[col_ordinales]
col_categoricas = list(set(col_no_numericas) - set(col_ordinales))
datos_categoricos = consumidores[col_categoricas]

Un pipeline de sklearn se define como una secuencia de pasos. Cada paso se define con una tupla de forma `[nombre del paso, transformador]`

Por ejemplo, si queremos crear un pipeline que procese las variables numéricas, primero imputándolas y después estandarizandolas, podriamos crear un pipeline

Creamos ahora los pipelines para cada tipo de variable. Es necesario aclarar que se va ha hacer uso de la libreria ColumnTransformer para el procesamiento de variables categoricas y categoricas ordinales

In [8]:
transformador_numerico = Pipeline([
    ("imputador_numerico", SimpleImputer(missing_values=np.nan, strategy='mean')),
    ("escalador_numerico", preprocessing.MinMaxScaler())
])

pipeline_numerico = ColumnTransformer([
    ("transformador_numerico", transformador_numerico, col_numericas),
])

pipeline_numerico.fit_transform(consumidores)[:5]

array([[0.30769231, 0.        ],
       [0.23076923, 0.25      ],
       [0.23076923, 0.16666667],
       [0.38461538, 0.        ],
       [0.23076923, 0.08333333]])

In [9]:
pipeline_numerico

ColumnTransformer(transformers=[('transformador_numerico',
                                 Pipeline(steps=[('imputador_numerico',
                                                  SimpleImputer()),
                                                 ('escalador_numerico',
                                                  MinMaxScaler())]),
                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])

Ahora se procede a crear el pipeline categorico

In [10]:
transformador_categorico = ColumnTransformer([
    ("transformador_categorico", OneHotEncoder(handle_unknown="impute"), col_categoricas)
])

pipeline_categorico = Pipeline([
    ("transformador_categorico", transformador_categorico)
])

pipeline_categorico.fit_transform(consumidores)[:5]

array([[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
        0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1,
        0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [11]:
pipeline_categorico

Pipeline(steps=[('transformador_categorico',
                 ColumnTransformer(transformers=[('transformador_categorico',
                                                  OneHotEncoder(handle_unknown='impute'),
                                                  ['g_01_familiares_consumen_sp_tipo',
                                                   'departamento',
                                                   'vivienda_tipo',
                                                   'd2_05_nivel_educativo_tipo',
                                                   'g_08_b_ofrecieron_cocaina_imp_tipo',
                                                   'd_12_c_presenta_problema_expendio_sp_barrio_tipo',
                                                   'situacion_tipo',
                                                   'g_05_...
                                                   'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                   'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                   'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                   'g_02_amigos_consumen_sp_tipo',
                                                   'g_08_c_ofrecieron_basuco_imp_tipo',
                                                   'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                   'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                   'd_09_deprimido_tipo',
                                                   'per_sexo_tipo', ...])]))])

`ColumnTransformer` requiere un mapeado específico para cada columna, indicando el número por el que queremos reemplazar cada nivel de cada variable ordinal. Si no especificamos un mapa, asume que el orden es aleatorio.

In [12]:
niveles_ordinales = []
for col, levels in dict_var_ordinales.items():
    niveles_ordinales.append({"col": col, 
                              "mapping": dict(zip(levels, range(len(levels))))})

In [13]:
transformador_ordinal = ColumnTransformer([
    ("transformador_ordinal", OrdinalEncoder(mapping=niveles_ordinales), col_ordinales)
])

pipeline_ordinal = Pipeline([
    ("transformador_ordinal", transformador_ordinal)
])
pipeline_ordinal.fit_transform(consumidores)[:5]

array([[1, 0, 1, 0, 0, 0],
       [2, 0, 2, 0, 0, 0],
       [2, 0, 3, 0, 0, 0],
       [2, 2, 3, 0, 0, 0],
       [1, 0, 3, 0, 0, 0]])

Ya tenemos una manera de, dado un conjunto de datos, separarlos y aplicar distintas transformaciones a cada variable. Nos falta una manera de, una vez se han transformado, reunirlas de nuevo.

Para ello podemos usar `FeatureUnion`, que simplemente toma un conjunto de pasos de un pipeline y los une.

In [14]:
pipeline_procesado = FeatureUnion([
    ('variables_numericas', pipeline_numerico),
    ('variables_ordinales', pipeline_ordinal),
    ('variables_categoricas', pipeline_categorico),
])

In [15]:
pipeline_procesado

FeatureUnion(transformer_list=[('variables_numericas',
                                ColumnTransformer(transformers=[('transformador_numerico',
                                                                 Pipeline(steps=[('imputador_numerico',
                                                                                  SimpleImputer()),
                                                                                 ('escalador_numerico',
                                                                                  MinMaxScaler())]),
                                                                 Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                               ('variables_ordinales',
                                Pipeline(steps=[('transformador_ordinal',
                                                 ColumnTransfor...
                                                                                   'g_01_a_num_familiares_consumen_sp_imp_tipo',
                                                                                   'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                   'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                   'g_02_amigos_consumen_sp_tipo',
                                                                                   'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                   'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                   'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                   'd_09_deprimido_tipo',
                                                                                   'per_sexo_tipo', ...])]))]))])

In [16]:
consumidores_procesado = pipeline_procesado.fit_transform(consumidores)

In [17]:
consumidores_procesado

array([[0.30769231, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.23076923, 0.25      , 2.        , ..., 0.        , 0.        ,
        0.        ],
       [0.23076923, 0.16666667, 2.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.23076923, 0.25      , 3.        , ..., 0.        , 0.        ,
        0.        ],
       [0.23076923, 0.25      , 2.        , ..., 0.        , 0.        ,
        0.        ],
       [0.23076923, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

Finalmente, necesitamos añadir un estimador al final para predecir en base a los datos transformados, pero primero se separa los datos en datos de entrenamiento y datos de test

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
     consumidores[variables_independientes], consumidores["CatRiesgo"],
     test_size=0.20, random_state=42
)

In [19]:
# Importación de las bibliotecas necesarias
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
estimador = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

In [20]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8000, 39) (8000,)
(2000, 39) (2000,)


Se debe realizar la separacion de datos en datos de entrenamiento y datos de test y ademas se debe establecer el estimador a usar en el modelo

In [21]:
pipeline_estimador = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', estimador)
])

In [22]:
pipeline_estimador.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'g_02_amigos_consumen_sp_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador', GradientBoostingClassifier(random_state=42))])

In [23]:
predicciones = pipeline_estimador.predict(X_test)

Se evalua el modelo segun lo obtenido

El beneficio de los pipelines, no solo es tener codigo mas legible y poder gestionar de forma ordenada todo el ciclo de vida del modelado, sino que los pipelines tienen todos los beneficios de los objetos de scikitlearn, por ejemplo, podemos usar validacion cruzada directamente con el pipeline.

In [28]:
resultados = {}
def exactitud_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.accuracy_score(y, preds)

def precision_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.precision_score(y, preds, average="macro")

def exhaustividad_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.recall_score(y, preds, average="macro")

def f1_multietiqueta(estimador, X, y):
    preds = estimador.predict(X)
    return metrics.f1_score(y, preds, average="macro")

In [29]:
def evaluar_modelo(prmPipeline,X, y, semillas):
    restultados_estimador = {}
    restultados_estimador["exactitud"] = cross_validate(prmPipeline, X, y,
                                           scoring=exactitud_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["precision"] = cross_validate(prmPipeline, X, y,
                                           scoring=precision_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["exhaustividad"] = cross_validate(prmPipeline, X, y,
                                           scoring=exhaustividad_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    
    restultados_estimador["f1_score"] = cross_validate(prmPipeline, X, y,
                                           scoring=f1_multietiqueta, n_jobs=-1, cv=semillas, return_train_score=True)
    return restultados_estimador

In [30]:
#Funcion para poder visuallizar los resultados de las metricas
def ver_resultados():
    df_res = pd.DataFrame()
    for modelo, metricas in resultados.items():
        for metrica, valores in metricas.items():
            for medida, arreglo in valores.items():
                # Calcula la media de cada array y agrega una nueva fila al DataFrame
                df_res.loc[f"{modelo}_{metrica}", medida] = np.mean(arreglo)
                
    return df_res

In [31]:
estimador_agregado = pipeline_estimador.named_steps['estimador']
resultados["gradientboosting"] = evaluar_modelo(estimador_agregado, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
gradientboosting_exactitud,77.850725,0.006250,0.955300,0.976892
gradientboosting_precision,76.215943,0.007506,0.941037,0.976929
gradientboosting_exhaustividad,76.946407,0.007810,0.941058,0.976324
gradientboosting_f1_score,76.369299,0.008123,0.939251,0.976610


In [32]:
report = classification_report(y_test, predicciones)
print('Reporte de Clasificación:')
print(report)

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       864
           1       0.93      0.94      0.93       477
           2       0.94      0.93      0.94       475
           3       0.90      0.84      0.87       114
           4       0.91      0.96      0.93        70

    accuracy                           0.95      2000
   macro avg       0.93      0.93      0.93      2000
weighted avg       0.95      0.95      0.95      2000



### Optimizacion de hiperparametros
En esta sección se configuran los diferentes parámetros del modelo, con el objetivo de mejorar los resultados de las métricas.

In [33]:
print(estimador.__doc__)

Gradient Boosting for classification.

    This algorithm builds an additive model in a forward stage-wise fashion; it
    allows for the optimization of arbitrary differentiable loss functions. In
    each stage ``n_classes_`` regression trees are fit on the negative gradient
    of the loss function, e.g. binary or multiclass log loss. Binary
    classification is a special case where only a single regression tree is
    induced.

    :class:`sklearn.ensemble.HistGradientBoostingClassifier` is a much faster
    variant of this algorithm for intermediate datasets (`n_samples >= 10_000`).

    Read more in the :ref:`User Guide <gradient_boosting>`.

    Parameters
    ----------
    loss : {'log_loss', 'deviance', 'exponential'}, default='log_loss'
        The loss function to be optimized. 'log_loss' refers to binomial and
        multinomial deviance, the same as used in logistic regression.
        It is a good choice for classification with probabilistic outputs.
        For loss '

In [38]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [39]:
param_distributions = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

In [40]:
random_search = RandomizedSearchCV(estimator=gradient_boosting_model, param_distributions=param_distributions, n_iter=100, cv=3, n_jobs=-1, verbose=2, random_state=42)

In [41]:
pipeline_estimador_op = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search)
])

In [42]:
pipeline_estimador_op.fit(X_train, y_train)

C:\Users\jefer\.conda\envs\Data-tesis\Lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 27 is smaller than n_iter=100. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 27 candidates, totalling 81 fits


Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(cv=3,
                                    estimator=GradientBoostingClassifier(random_state=42),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'learning_rate': [0.01,
                                                                           0.1,
                                                                           0.2],
                                                         'max_depth': [3, 4, 5],
                                                         'n_estimators': [50,
                                                                          100,
                                                                          200]},
                                    random_state=42, verbose=2))])

In [44]:
best_params = random_search.best_params_

In [45]:
pipeline_estimador_op.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

Se crea un pipeline con el mejor estimador obtenido

In [46]:
pipeline_estimador_op_best = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search.best_estimator_)
])

In [47]:
pipeline_estimador_op_best.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'd_11_k_conocimiento_riesgo_cocaina_frecuentemente_tipo',
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'g_02_amigos_consumen_sp_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            random_state=42))])

In [48]:
pipeline_estimador_op_best.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [49]:
estimador_best = pipeline_estimador_op_best.named_steps['estimador']
resultados["grboost_best"] = evaluar_modelo(estimador_best, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
gradientboosting_exactitud,77.850725,0.006250,0.955300,0.976892
gradientboosting_precision,76.215943,0.007506,0.941037,0.976929
gradientboosting_exhaustividad,76.946407,0.007810,0.941058,0.976324
gradientboosting_f1_score,76.369299,0.008123,0.939251,0.976610
grboost_best_exactitud,75.032296,0.007336,0.961400,0.993290
grboost_best_precision,72.589632,0.008435,0.947881,0.994258
grboost_best_exhaustividad,72.668780,0.007810,0.945195,0.993267
grboost_best_f1_score,73.312963,0.009060,0.944883,0.993757


### Segunda optimizacion

In [52]:
param_distributions_2 = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 6, 9],
    'subsample': [0.8, 0.9, 1.0],
}

In [53]:
random_search_2 = RandomizedSearchCV(
    estimator=GradientBoostingClassifier(random_state=42), 
    param_distributions=param_distributions_2, 
    n_iter=50, 
    cv=3, 
    n_jobs=-1, 
    verbose=2, 
    random_state=42
)

In [56]:
pipeline_estimador_op2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_2)
])

In [57]:
pipeline_estimador_op2.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(cv=3,
                                    estimator=GradientBoostingClassifier(random_state=42),
                                    n_iter=50, n_jobs=-1,
                                    param_distributions={'learning_rate': [0.01,
                                                                           0.05,
                                                                           0.1],
                                                         'max_depth': [3, 6, 9],
                                                         'n_estimators': [50,
                                                                          100,
                                                                          150],
                                                         'subsample': [0.8, 0.9,
                                                                       1.0]},
                                    random_state=42, verbose=2))])

In [58]:
best_params2 = random_search_2.best_params_

In [59]:
pipeline_estimador_op2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [60]:
pipeline_estimador_op_best2 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', random_search_2.best_estimator_)
])

In [61]:
pipeline_estimador_op_best2.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'g_02_amigos_consumen_sp_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador',
                 GradientBoostingClassifier(n_estimators=150, random_state=42,
                                            subsample=0.9))])

In [62]:
pipeline_estimador_op_best2.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [63]:
estimador_best2 = pipeline_estimador_op_best2.named_steps['estimador']
resultados["grboost_best_2"] = evaluar_modelo(estimador_best2, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
gradientboosting_exactitud,77.850725,0.006250,0.955300,0.976892
gradientboosting_precision,76.215943,0.007506,0.941037,0.976929
gradientboosting_exhaustividad,76.946407,0.007810,0.941058,0.976324
gradientboosting_f1_score,76.369299,0.008123,0.939251,0.976610
grboost_best_exactitud,75.032296,0.007336,0.961400,0.993290
grboost_best_precision,72.589632,0.008435,0.947881,0.994258
grboost_best_exhaustividad,72.668780,0.007810,0.945195,0.993267
grboost_best_f1_score,73.312963,0.009060,0.944883,0.993757
grboost_best_2_exactitud,99.089514,0.006885,0.960500,0.988053
grboost_best_2_precision,97.858989,0.009301,0.946311,0.989055


### Tercera optimizacion

In [67]:
param_dist_random3 = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [68]:
busqueda_random3 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_dist_random3,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [69]:
pipeline_estimador_op3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random3)
])

In [70]:
pipeline_estimador_op3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(estimator=GradientBoostingClassifier(random_state=42),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'learning_rate': [0.01,
                                                                           0.05,
                                                                           0.1],
                                                         'max_depth': [3, 5, 7],
                                                         'min_samples_leaf': [1,
                                                                              2,
                                                                              4],
                                                         'min_samples_split': [2,
                                                                               5,
                                                                               10],
                                                         'n_estimators': [50,
                                                                          100,
                                                                          200]},
                                    scoring='accuracy'))])

In [71]:
pipeline_estimador_op3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [72]:
pipeline_estimador_op_best3 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random3.best_estimator_)
])

In [73]:
pipeline_estimador_op_best3.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'g_02_amigos_consumen_sp_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador',
                 GradientBoostingClassifier(min_samples_leaf=2,
                                            n_estimators=200,
                                            random_state=42))])

In [75]:
pipeline_estimador_op_best3.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [74]:
estimador_best3 = pipeline_estimador_op_best3.named_steps['estimador']
resultados["grboost_best_3"] = evaluar_modelo(estimador_best3, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
gradientboosting_exactitud,77.850725,0.006250,0.955300,0.976892
gradientboosting_precision,76.215943,0.007506,0.941037,0.976929
gradientboosting_exhaustividad,76.946407,0.007810,0.941058,0.976324
gradientboosting_f1_score,76.369299,0.008123,0.939251,0.976610
grboost_best_exactitud,75.032296,0.007336,0.961400,0.993290
grboost_best_precision,72.589632,0.008435,0.947881,0.994258
grboost_best_exhaustividad,72.668780,0.007810,0.945195,0.993267
grboost_best_f1_score,73.312963,0.009060,0.944883,0.993757
grboost_best_2_exactitud,99.089514,0.006885,0.960500,0.988053
grboost_best_2_precision,97.858989,0.009301,0.946311,0.989055


### Cuarta optimizacion

In [76]:
param_distributions_4 = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'max_features': ['auto', 'sqrt', 'log2']
}

In [77]:
busqueda_random4 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_distributions_4,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [78]:
pipeline_estimador_op4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random4)
])

In [79]:
pipeline_estimador_op4.fit(X_train, y_train)

C:\Users\jefer\.conda\envs\Data-tesis\Lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 81 is smaller than n_iter=100. Running 81 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(estimator=GradientBoostingClassifier(random_state=42),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'learning_rate': [0.01,
                                                                           0.1,
                                                                           0.2],
                                                         'max_depth': [3, 4, 5],
                                                         'max_features': ['auto',
                                                                          'sqrt',
                                                                          'log2'],
                                                         'n_estimators': [100,
                                                                          200,
                                                                          300]},
                                    scoring='accuracy'))])

In [81]:
best_params1 = busqueda_random4.best_params_

In [82]:
pipeline_estimador_op4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [83]:
pipeline_estimador_op_best4 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random4.best_estimator_)
])

In [84]:
pipeline_estimador_op_best4.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_06_c_posibilidad_conseguir_basuco_tipo',
                                                                                                    'g_02_amigos_consumen_sp_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador',
                 GradientBoostingClassifier(learning_rate=0.2,
                                            max_features='sqrt',
                                            n_estimators=300,
                                            random_state=42))])

In [85]:
pipeline_estimador_op_best4.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [86]:
estimador_best4 = pipeline_estimador_op_best4.named_steps['estimador']
resultados["grboost_best_4"] = evaluar_modelo(estimador_best4, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
gradientboosting_exactitud,77.850725,0.006250,0.955300,0.976892
gradientboosting_precision,76.215943,0.007506,0.941037,0.976929
gradientboosting_exhaustividad,76.946407,0.007810,0.941058,0.976324
gradientboosting_f1_score,76.369299,0.008123,0.939251,0.976610
grboost_best_exactitud,75.032296,0.007336,0.961400,0.993290
grboost_best_precision,72.589632,0.008435,0.947881,0.994258
grboost_best_exhaustividad,72.668780,0.007810,0.945195,0.993267
grboost_best_f1_score,73.312963,0.009060,0.944883,0.993757
grboost_best_2_exactitud,99.089514,0.006885,0.960500,0.988053
grboost_best_2_precision,97.858989,0.009301,0.946311,0.989055


In [82]:
resultadoEvaluacionOPBest4

{'exactitud': 0.9429999999999998,
 'precision': 0.9454240644434684,
 'exhaustividad': 0.9447,
 'f1_score': 0.9429000000000001}

### Quinta optimizacion

In [87]:
param_distributions_5 = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'max_features': [None, 'sqrt', 'log2']
}

In [88]:
busqueda_random5 = RandomizedSearchCV(estimator=estimador, 
                    param_distributions=param_distributions_5,
                   scoring="accuracy", n_jobs=-1, n_iter=100)

In [89]:
pipeline_estimador_op5 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random5)
])

In [90]:
pipeline_estimador_op5.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador',
                 RandomizedSearchCV(estimator=GradientBoostingClassifier(random_state=42),
                                    n_iter=100, n_jobs=-1,
                                    param_distributions={'learning_rate': [0.05,
                                                                           0.1,
                                                                           0.15],
                                                         'max_depth': [3, 5, 7],
                                                         'max_features': [None,
                                                                          'sqrt',
                                                                          'log2'],
                                                         'n_estimators': [50,
                                                                          100,
                                                                          200],
                                                         'subsample': [0.8, 0.9,
                                                                       1.0]},
                                    scoring='accuracy'))])

In [91]:
pipeline_estimador_op5.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [93]:
print(busqueda_random.best_score_)
print(busqueda_random.best_estimator_)

NameError: name 'busqueda_random' is not defined

In [94]:
print(busqueda_random5.best_score_)
print(busqueda_random5.best_estimator_)

0.959375
GradientBoostingClassifier(learning_rate=0.15, max_features='sqrt',
                           n_estimators=200, random_state=42, subsample=0.8)


In [95]:
pipeline_estimador_op_best5 = Pipeline([
    ('procesador', pipeline_procesado),
    ('estimador', busqueda_random5.best_estimator_)
])

In [96]:
pipeline_estimador_op_best5.fit(X_train, y_train)

Pipeline(steps=[('procesador',
                 FeatureUnion(transformer_list=[('variables_numericas',
                                                 ColumnTransformer(transformers=[('transformador_numerico',
                                                                                  Pipeline(steps=[('imputador_numerico',
                                                                                                   SimpleImputer()),
                                                                                                  ('escalador_numerico',
                                                                                                   MinMaxScaler())]),
                                                                                  Index(['residentes_hogar', 'd2_04_num_hijos'], dtype='object'))])),
                                                ('variables_ordinales',
                                                 Pipeline(steps=[('transfo...
                                                                                                    'g_02_amigos_consumen_sp_tipo',
                                                                                                    'g_08_c_ofrecieron_basuco_imp_tipo',
                                                                                                    'd_11_h_conocimiento_riesgo_fumar_marihuana_frecuentemente_tipo',
                                                                                                    'g_08_a_ofrecieron_marihuana_imp_tipo',
                                                                                                    'd_09_deprimido_tipo',
                                                                                                    'per_sexo_tipo', ...])]))]))])),
                ('estimador',
                 GradientBoostingClassifier(learning_rate=0.15,
                                            max_features='sqrt',
                                            n_estimators=200, random_state=42,
                                            subsample=0.8))])

In [97]:
pipeline_estimador_op_best5.predict(X_test)

array([0, 2, 0, ..., 2, 0, 4], dtype=int64)

In [98]:
estimador_best5 = pipeline_estimador_op_best5.named_steps['estimador']
resultados["grboost_best_5"] = evaluar_modelo(estimador_best5, consumidores_procesado, consumidores.CatRiesgo, 50)
ver_resultados()

,fit_time,score_time,test_score,train_score
gradientboosting_exactitud,77.850725,0.006250,0.955300,0.976892
gradientboosting_precision,76.215943,0.007506,0.941037,0.976929
gradientboosting_exhaustividad,76.946407,0.007810,0.941058,0.976324
gradientboosting_f1_score,76.369299,0.008123,0.939251,0.976610
grboost_best_exactitud,75.032296,0.007336,0.961400,0.993290
grboost_best_precision,72.589632,0.008435,0.947881,0.994258
grboost_best_exhaustividad,72.668780,0.007810,0.945195,0.993267
grboost_best_f1_score,73.312963,0.009060,0.944883,0.993757
grboost_best_2_exactitud,99.089514,0.006885,0.960500,0.988053
grboost_best_2_precision,97.858989,0.009301,0.946311,0.989055


In [93]:
resultadoEvaluacionOPBest5 = evaluar_modelo_cross(pipeline_estimador_op_best5,consumidores.drop('CatRiesgo', axis=1), consumidores.CatRiesgo, 50)

In [94]:
resultadoEvaluacionOPBest5

{'exactitud': 0.9442,
 'precision': 0.9445854075315496,
 'exhaustividad': 0.9441999999999999,
 'f1_score': 0.945}